In [24]:
import io
import os
import base64
import logging
import pandas as pd
import plotly.express as px

from data_ingestion.data_loader import DataLoader  # Replace with your actual import
from utilities.logging import setup_logger

# set the input file and output directory

In [25]:
input_file_path = "./data/000091_231213_153600.GDS.xlsx"
output_data_dir = "./data"

In [26]:
data_loader = DataLoader(input_file_path)
data_loader.auto_load(file_path=input_file_path)

c:\Users\ecountrywood\dev\tools\pdf_tools\env\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning:

Workbook contains no default style, apply openpyxl's default



# load the dataframe 

In [27]:
df = data_loader.data["data_frame"]

In [28]:
# save a copy of the original df
df_original = df.copy()

In [29]:
df_original.head()

,Date,Time,sec,TC1 Gasket,,TC2 Supply Lug,,TC3 Backup Lug,,TC4 Subpanel Lug,...,TC34 Cap on Holdup PCB,,TC36 TACO PCB,,TC31 Windings of Transformer,,TC35 Holdup PCB under Power,,TC33 Busbar L2 Neut to Relay,
0,2023/12/13,15:36:00,0,23.6,23.8,22.8,22.9,22.8,22.8,22.8,...,26.8,26.9,34.8,34.8,25.5,25.6,25.7,25.8,23.7,23.8
1,2023/12/13,15:38:00,0,23.5,23.6,22.8,24.1,22.8,23.0,22.9,...,26.9,27.1,31.6,34.8,25.5,25.9,25.8,26.1,23.7,24.3
2,2023/12/13,15:40:00,0,23.5,23.6,24.1,26.9,23.0,23.5,23.9,...,26.9,27.1,29.7,31.6,25.9,25.9,26.1,26.3,24.3,26.8
3,2023/12/13,15:42:00,0,23.6,23.6,26.9,29.8,23.5,24.3,25.7,...,26.9,27.0,29.3,29.7,25.9,25.9,26.2,26.4,26.8,30.1
4,2023/12/13,15:44:00,0,23.6,23.7,29.8,32.6,24.3,25.3,27.4,...,26.9,27.0,29.2,29.3,25.9,26.0,26.4,26.5,30.1,33.6


In [30]:

def replace_tc_columns(df):
    # keep the columns that do not contain TC
    # these are the MAX columns
    modified_df = df.copy()
    for col in modified_df.columns:
        if 'TC' in col:
            col_index = modified_df.columns.get_loc(col)
            if col_index + 1 < len(modified_df.columns):
                modified_df[col] = modified_df.iloc[:, col_index + 1]
    # drop the columns with empth sting as name
    modified_df = modified_df.drop('', axis=1)         
    return modified_df

# Usage
# adjusted_df = replace_tc_columns(your_input_dataframe)
df_original = replace_tc_columns(df_original)
# drop the sec column
df_original = df_original.drop('sec', axis=1)
df_original.head()

,Date,Time,TC1 Gasket,TC2 Supply Lug,TC3 Backup Lug,TC4 Subpanel Lug,TC5 MSA Busbar,TC6 Glass Door,TC7 Upper Enclosure Ambient,TC8 Lower Enclosure Ambient,...,TC27 Relay Ambient,TC28 MCB Ambient,TC29 Capacitor (C44),TC30 Winding Internal Side Trans,TC32 Busbar L1 Neut to Relay,TC34 Cap on Holdup PCB,TC36 TACO PCB,TC31 Windings of Transformer,TC35 Holdup PCB under Power,TC33 Busbar L2 Neut to Relay
0,2023/12/13,15:36:00,23.8,22.9,22.8,23.0,24.2,23.7,24.9,22.8,...,24.3,23.0,24.5,25.5,24.0,26.9,34.8,25.6,25.8,23.8
1,2023/12/13,15:38:00,23.6,24.1,23.0,23.9,29.0,23.6,26.4,22.9,...,25.1,23.5,24.8,25.6,24.5,27.1,34.8,25.9,26.1,24.3
2,2023/12/13,15:40:00,23.6,26.9,23.5,25.7,38.2,23.6,26.5,23.0,...,25.2,24.9,25.6,25.7,26.7,27.1,31.6,25.9,26.3,26.8
3,2023/12/13,15:42:00,23.6,29.8,24.3,27.4,45.9,23.6,26.5,23.2,...,25.4,26.6,27.0,25.7,29.7,27.0,29.7,25.9,26.4,30.1
4,2023/12/13,15:44:00,23.7,32.6,25.3,29.0,52.4,23.6,26.7,23.5,...,25.5,28.0,28.9,25.7,32.8,27.0,29.3,26.0,26.5,33.6


In [31]:
# plot all columns of the original df using plotly and display in this notebook
msa_columns = [col for col in df_original.columns if 'MSA' in col]

# fig = px.line(df_original, x=df_original['Time'], y=msa_columns, yaxis_title='Temperature (Degrees C)', title='Original Data')
fig = px.line(df_original, x='Time', y=msa_columns, title='Original Data', 
              labels={'value': 'Temperature (Degrees C)'})

fig.show()


# Process Data
## now we have a dataframe for the notebook work

1. split the data into two tests 25C and 50C
2. for each test, we need max temp of each TC
3. downsample the data into 15 minute intervals
4. combine downsampled data and TC max
5. Transpose the result
6. re-arrange the rows to match the order in the TDP

# Split data into two test periods
If data represents only one single test, set 'test_1' to 'df_original'

In [32]:


def split_dataframe_by_time_old(df, d1_start_time, d1_end_time, d2_start_time, d2_end_time, use_last_entry=False):
    # Convert 'Time' to timedelta for filtering
    df['Timedelta'] = pd.to_timedelta(df['Time'])

    # Convert input times to Timedelta
    d1_start_time = pd.to_timedelta(d1_start_time)
    d1_end_time = pd.to_timedelta(d1_end_time)
    d2_start_time = pd.to_timedelta(d2_start_time)
    d2_end_time = pd.to_timedelta(d2_end_time)

    # Filter rows based on time range
    df1 = df[(df['Timedelta'] >= d1_start_time) & (df['Timedelta'] <= d1_end_time)]
    if use_last_entry:
        df2 = df[(df['Timedelta'] >= d2_start_time)]
    else:
        df2 = df[(df['Timedelta'] >= d2_start_time) & (df['Timedelta'] <= d2_end_time)]
    
    return df1, df2


In [33]:

def split_dataframe_by_time(df, first_period_start, first_period_end, second_period_start, second_period_end=None, use_last_entry=False):
    """
    Splits a DataFrame into two based on specified time ranges. The end time for the second period is optional if 
    use_last_entry is True.
    
    Args:
        df (DataFrame): The DataFrame to split.
        first_period_start (str): Start time for the first period.
        first_period_end (str): End time for the first period.
        second_period_start (str): Start time for the second period.
        second_period_end (str, optional): End time for the second period. Default is None.
        use_last_entry (bool): If True, the second DataFrame includes all entries from the start time to the last entry.

    Returns:
        tuple: Two DataFrames split based on the specified time periods.
    """
    # Ensure 'Time' is in Timedelta format
    if not isinstance(df['Time'].iloc[0], pd.Timedelta):
        df['Timedelta'] = pd.to_timedelta(df['Time'])

    # Convert input times to Timedelta
    first_start_td = pd.to_timedelta(first_period_start)
    first_end_td = pd.to_timedelta(first_period_end)
    second_start_td = pd.to_timedelta(second_period_start)
    
    # Split the DataFrame for the first period
    df1 = df[(df['Timedelta'] >= first_start_td) & (df['Timedelta'] <= first_end_td)]

    # Split the DataFrame for the second period
    if use_last_entry:
        df2 = df[df['Timedelta'] >= second_start_td]
    else:
        if second_period_end is not None:
            second_end_td = pd.to_timedelta(second_period_end)
            df2 = df[(df['Timedelta'] >= second_start_td) & (df['Timedelta'] <= second_end_td)]
        else:
            raise ValueError("second_period_end must be specified if use_last_entry is False.")

    return df1, df2




# input start and end times of each test data set
test_1, test_2 = split_dataframe_by_time(df_original, 
                    first_period_start='15:38:00',
                    first_period_end='16:46:20',
                    second_period_start='16:46:20',
                    # if d2_end_time < d2_start_time, then d2_end_time = last entry in df
                    second_period_end='16:46:00',
                    use_last_entry=False 
                    )
test_1.shape, test_2.shape

((35, 39), (0, 39))

In [34]:
test_1.head()

,Date,Time,TC1 Gasket,TC2 Supply Lug,TC3 Backup Lug,TC4 Subpanel Lug,TC5 MSA Busbar,TC6 Glass Door,TC7 Upper Enclosure Ambient,TC8 Lower Enclosure Ambient,...,TC28 MCB Ambient,TC29 Capacitor (C44),TC30 Winding Internal Side Trans,TC32 Busbar L1 Neut to Relay,TC34 Cap on Holdup PCB,TC36 TACO PCB,TC31 Windings of Transformer,TC35 Holdup PCB under Power,TC33 Busbar L2 Neut to Relay,Timedelta
1,2023/12/13,15:38:00,23.6,24.1,23.0,23.9,29.0,23.6,26.4,22.9,...,23.5,24.8,25.6,24.5,27.1,34.8,25.9,26.1,24.3,0 days 15:38:00
2,2023/12/13,15:40:00,23.6,26.9,23.5,25.7,38.2,23.6,26.5,23.0,...,24.9,25.6,25.7,26.7,27.1,31.6,25.9,26.3,26.8,0 days 15:40:00
3,2023/12/13,15:42:00,23.6,29.8,24.3,27.4,45.9,23.6,26.5,23.2,...,26.6,27.0,25.7,29.7,27.0,29.7,25.9,26.4,30.1,0 days 15:42:00
4,2023/12/13,15:44:00,23.7,32.6,25.3,29.0,52.4,23.6,26.7,23.5,...,28.0,28.9,25.7,32.8,27.0,29.3,26.0,26.5,33.6,0 days 15:44:00
5,2023/12/13,15:46:00,23.8,35.3,26.4,30.5,57.9,23.6,26.9,23.8,...,29.2,31.2,25.8,35.8,27.3,29.5,26.2,26.7,36.9,0 days 15:46:00


In [35]:
def get_max_values(df):
    # # Find the max value for each column
    max_values = df.max()

    max_values_df = pd.DataFrame([max_values])

    return max_values_df

max_values_df_test_1 = get_max_values(test_1)
# max_values_df_test_2 = get_max_values(test_2)


# Downsample to get 15 minute intervals

In [36]:
# Usage
test_1_15_min = test_1.iloc[::7, :]
test_1_15_min.head()

,Date,Time,TC1 Gasket,TC2 Supply Lug,TC3 Backup Lug,TC4 Subpanel Lug,TC5 MSA Busbar,TC6 Glass Door,TC7 Upper Enclosure Ambient,TC8 Lower Enclosure Ambient,...,TC28 MCB Ambient,TC29 Capacitor (C44),TC30 Winding Internal Side Trans,TC32 Busbar L1 Neut to Relay,TC34 Cap on Holdup PCB,TC36 TACO PCB,TC31 Windings of Transformer,TC35 Holdup PCB under Power,TC33 Busbar L2 Neut to Relay,Timedelta
1,2023/12/13,15:38:00,23.6,24.1,23.0,23.9,29.0,23.6,26.4,22.9,...,23.5,24.8,25.6,24.5,27.1,34.8,25.9,26.1,24.3,0 days 15:38:00
8,2023/12/13,15:52:00,24.4,42.3,29.9,34.5,70.7,23.9,28.0,24.9,...,31.7,38.7,26.2,44.0,28.3,30.9,26.9,27.5,46.0,0 days 15:52:00
15,2023/12/13,16:06:00,26.7,54.3,37.4,42.0,90.0,25.3,31.8,27.7,...,37.1,56.0,27.9,59.4,32.3,35.7,29.5,30.6,62.2,0 days 16:06:00
22,2023/12/13,16:20:00,29.3,62.4,43.3,47.9,102.8,27.4,35.8,30.1,...,41.3,69.6,30.0,70.9,36.5,40.3,32.1,34.1,73.9,0 days 16:20:00
29,2023/12/13,16:34:00,31.9,68.2,48.1,52.7,112.2,29.6,39.4,32.4,...,44.0,80.1,32.6,79.9,40.5,44.5,35.0,37.4,82.6,0 days 16:34:00


In [37]:

# Concatenate test max values this with the original DataFrame
test_1_df = pd.concat([test_1_15_min, max_values_df_test_1], ignore_index=True)
# test_2_df = pd.concat([test_2_15_min, max_values_df_test_2], ignore_index=True)
test_1_df.head()

,Date,Time,TC1 Gasket,TC2 Supply Lug,TC3 Backup Lug,TC4 Subpanel Lug,TC5 MSA Busbar,TC6 Glass Door,TC7 Upper Enclosure Ambient,TC8 Lower Enclosure Ambient,...,TC28 MCB Ambient,TC29 Capacitor (C44),TC30 Winding Internal Side Trans,TC32 Busbar L1 Neut to Relay,TC34 Cap on Holdup PCB,TC36 TACO PCB,TC31 Windings of Transformer,TC35 Holdup PCB under Power,TC33 Busbar L2 Neut to Relay,Timedelta
0,2023/12/13,15:38:00,23.6,24.1,23.0,23.9,29.0,23.6,26.4,22.9,...,23.5,24.8,25.6,24.5,27.1,34.8,25.9,26.1,24.3,0 days 15:38:00
1,2023/12/13,15:52:00,24.4,42.3,29.9,34.5,70.7,23.9,28.0,24.9,...,31.7,38.7,26.2,44.0,28.3,30.9,26.9,27.5,46.0,0 days 15:52:00
2,2023/12/13,16:06:00,26.7,54.3,37.4,42.0,90.0,25.3,31.8,27.7,...,37.1,56.0,27.9,59.4,32.3,35.7,29.5,30.6,62.2,0 days 16:06:00
3,2023/12/13,16:20:00,29.3,62.4,43.3,47.9,102.8,27.4,35.8,30.1,...,41.3,69.6,30.0,70.9,36.5,40.3,32.1,34.1,73.9,0 days 16:20:00
4,2023/12/13,16:34:00,31.9,68.2,48.1,52.7,112.2,29.6,39.4,32.4,...,44.0,80.1,32.6,79.9,40.5,44.5,35.0,37.4,82.6,0 days 16:34:00


In [40]:
def transpose_15_min_and_max(df):
        # Transpose the 15 monute DataFrame
    transposed_df = df.T

    # Reset index to get the column names as a separate column
    transposed_df.reset_index(inplace=True)

    # Rename columns
    new_column_names = ['NAME'] + [f'x{i*15}' for i in range(transposed_df.shape[1] - 1)]
    transposed_df.columns = new_column_names
    
    # Get the name of the last column
    last_column_name = transposed_df.columns[-1]

    # Rename the last column to 'Max'    
    transposed_df.rename(columns={last_column_name: 'Max'}, inplace=True)
    return transposed_df

test_1_unordered_df = transpose_15_min_and_max(test_1_df)
test_1_unordered_df.head()




,NAME,x0,x15,x30,x45,x60,Max
0,Date,2023/12/13,2023/12/13,2023/12/13,2023/12/13,2023/12/13,2023/12/13
1,Time,15:38:00,15:52:00,16:06:00,16:20:00,16:34:00,16:46:00
2,TC1 Gasket,23.6,24.4,26.7,29.3,31.9,36.1
3,TC2 Supply Lug,24.1,42.3,54.3,62.4,68.2,70.7
4,TC3 Backup Lug,23.0,29.9,37.4,43.3,48.1,51.0


# re-arrange rows to match TDP

In [46]:
# get the original name list
original_name_list = test_1_unordered_df['NAME'].tolist()

# write to file to manually edit the order of the names
make_new_name_list = True # overwrite the name_list.txt file

if make_new_name_list:
    with open(os.path.join(output_data_dir, 'name_list.txt'), 'w') as f:
        for name in original_name_list:
            f.write(f'{name}\n')

new_order = [
    "Date",
    "Time",
    "TC1 Gasket",
    "TC2 Supply Lug",
    "TC3 Backup Lug",
    "TC4 Subpanel Lug",
    "TC5 MSA Busbar",
    "TC6 Glass Door",
    "TC7 Upper Enclosure Ambient",
    "TC8 Lower Enclosure Ambient",
    "TC10 Subpanel Tray",
    "TC11 Ambient under MSA Cover",
    "TC12 MSA PCB",
    "TC13 MCB Plastic",
    "TC14 Enclosure Handle",
    "TC15 Top of Enclosure (Plastic)",
    "TC16 Side of Enclosure",
    "TC17 Subpanel Bussing",
    "TC18 Branch Breaker Plastic Top",
    "TC19 Branch Breaker Plastic Mid",
    "TC20 Branch Breaker Bussing Top",
    "TC21 Brach Breaker Bussing Mid",
    "TC22 Transformer Cover Plastic",
    "TC23 RF Window Plastic",
    "TC24 Busbar Tray near MCB",
    "TC25 Busbar Tray Field Wiring",
    "TC26 Busbar Tray MSA Under cover",
    "TC27 Relay Ambient",
    "TC28 MCB Ambient",
    "TC9 Lab Ambient",
    "TC29 Capacitor (C44)",
    "TC36 TACO PCB",
    "TC30 Winding Internal Side Trans",
    "TC31 Windings of Transformer",
    "TC32 Busbar L1 Neut to Relay",
    "TC33 Busbar L2 Neut to Relay",
    "TC34 Cap on Holdup PCB",
    "TC35 Holdup PCB under Power",
]


In [47]:

# Reorder the DataFrame based on the new order
# Make sure all names in 'new_order' exist in 'df['NAME']'
test_1_final_df = test_1_unordered_df.set_index('NAME').loc[new_order].reset_index()

# test if the order is correct against the order in name_list
for i in range(len(new_order)):
    if new_order[i] != test_1_final_df['NAME'].tolist()[i]:
        print(f"ERROR: {new_order[i]} != {test_1_final_df['NAME'].tolist()[i]}")
        break

In [ ]:
test_1_final_df.head(45)

,NAME,x0,x15,x30,x45,x60,Max
0,Date,2023/12/06,2023/12/06,2023/12/06,2023/12/06,2023/12/06,2023/12/06
1,Time,17:06:40,17:22:00,17:37:20,17:52:40,18:08:00,18:19:20
2,TC1 Gasket,29.6,26.5,26.8,26.9,27.0,29.6
3,TC2 Supply Lug,28.9,46.7,55.9,59.9,62.3,63.4
4,TC3 Backup Lug,29.3,32.2,36.2,39.1,41.0,42.0
5,TC4 Subpanel Lug,29.3,40.7,47.3,51.7,54.6,55.6
6,TC5 MSA Busbar,34.6,65.4,81.0,90.8,97.2,99.7
7,TC6 Glass Door,28.9,26.7,26.8,26.8,26.9,28.9
8,TC7 Upper Enclosure Ambient,31.4,35.5,43.2,46.7,48.8,50.3
9,TC8 Lower Enclosure Ambient,28.9,29.6,31.2,32.2,32.9,33.1


# Generate output files with processed data

In [49]:
# generate output file names for processed data
def generate_output_filename(file_path, data_dir,temp):
    # Extract the base name of the file without the extension
    base_name = file_path.split('/')[-1].split('.')[0]
    # trim leading zeros from base name
    base_name = base_name.lstrip('0')
    
    # Construct the new file name with the .csv extension
    output_file = f"{data_dir}/processed_temp_data/{base_name}_{temp}_15min.csv"

    return output_file

test_1_output_file = generate_output_filename(input_file_path, output_data_dir,'25C')
# test_2_output_file = generate_output_filename(input_file_path, output_data_dir,'50C')

print(test_1_output_file)


./data/processed_temp_data/91_231213_153600_25C_15min.csv


# write processed data to csv

In [50]:
# write processed data to csv
test_1_final_df.to_csv(test_1_output_file, index = None, header=True)
# test_2_final_df.to_csv(test_2_output_file, index = None, header=True)